In [1]:
############ libraries ##################################
import cv2
import os
import numpy as np
import PIL
from scipy import misc
import imageio
from time import sleep
import time
from keras.models import model_from_json
import pandas as pd
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#%matplotlib inline
import math
import datetime
import tensorflow.keras as tf
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import BatchNormalization
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, LSTM, merge
from keras import metrics
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
from keras import layers
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
#from tensorflow.contrib.keras import losses,optimizers,metrics
#from tensorflow.contrib.keras import layers
from keras import initializers
from sklearn.model_selection import train_test_split
from sklearn.utils.multiclass import unique_labels
from keras.applications import VGG19
from keras.callbacks import ModelCheckpoint
import ipdb
import h5py
from keras.layers import LeakyReLU
from keras.models import load_model
from keras.models import Model
import json
from matplotlib import pyplot as plt
import sys
sys.path
sys.executable
import operator
import queue
import pickle
from tensorflow.keras.utils import Sequence
from keras.layers import Input
%matplotlib inline
import seaborn as sn
from sklearn.metrics import classification_report
from tensorflow.keras.applications.resnet50 import ResNet50
start = datetime.datetime.now()
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from PIL import Image, ImageOps
import numpy as np
from skimage import transform
from sklearn.preprocessing import LabelBinarizer
from keras.layers import concatenate
import glob


In [6]:
data_path = "Excel_data/input_fusion_paths_rough.csv"
root_dir="WOOD_PLASTIC/IMAGES_dummy"
global n_classes
n_classes=11
rv=[]#### return values object list from rough_data values
label=[]
image_list = []
a=[]
batch_size = 16
inception_resnet_v2 = applications.InceptionResNetV2(include_top=False, weights='imagenet')
datagen = ImageDataGenerator(rescale=1. / 255,rotation_range=90)
img_width, img_height = 128, 512



def load_samples(csv_file):
    global train_whole, test_whole, tr_rough_X, ts_rough_X, tr_img_X, ts_img_X 
    global  tr_rough_Y, ts_rough_Y, tr_img_Y, ts_img_Y
    ### use global inside a function to access those variables outside the function
    data = pd.read_csv(csv_file)
    data = data[['RaX','RaY','RqX','RqY','RzX','RzY','Filenames','Class','Labels']]
    training_data, testing_data = train_test_split(data, test_size=0.2, random_state=25)
    train_whole= np.array(training_data)
    test_whole =np.array(testing_data)
    tr_rough_X =train_whole[:,0:6] ## values train
    ts_rough_X =test_whole[:,0:6]  ## values test
    tr_img_X   =train_whole[:,6:9] ## imgs train, training data
    ts_img_X   =test_whole[:,6:9]  ## imgs test, validation data
    train_rough_Y =train_whole[:,7].reshape(-1,1)######## shape is (275,1)
    encoder = OneHotEncoder(sparse=False)
    tr_rough_Y = encoder.fit_transform(train_rough_Y)
    test_rough_Y =test_whole[:,7].reshape(-1,1)
    encoder = OneHotEncoder(sparse=False)
    ts_rough_Y = encoder.fit_transform(test_rough_Y)
    tr_img_Y   =train_whole[:,8] ## train labels
    ts_img_Y   =test_whole[:,8] ## test labels
    file_names = list(data.iloc[:,6])
    # Get the labels present in the second column
    labels = list(data.iloc[:,8])
    samples=[]
    for samp,lab in zip(file_names,labels):
        samples.append([samp,lab])
    return samples

samples = load_samples(data_path)
train_df = pd.DataFrame(tr_img_X) ## convert array to df
train_df.columns=['filename','class','labels'] ## add header to df
#print(train_df)
test_df = pd.DataFrame(ts_img_X) ## convert array to df
test_df.columns=['filename','class','labels'] ## add header filename to df
#print(test_df)


def images():
    
    ##Train#####################################################################################
    generator_train = datagen.flow_from_dataframe(train_df, directory=None, x_col='filename', y_col='class',target_size =(img_width, img_height), batch_size=batch_size, 
                      class_mode = 'categorical', shuffle=False) 
    nb_train_samples = len(train_df)
    print('nb_train_samples', nb_train_samples)
    predict_size_train = int(math.ceil(nb_train_samples / batch_size)) 
    bottleneck_features_train = inception_resnet_v2.predict_generator(generator_train, predict_size_train) 
    np.save("features_training_inception_fusion_seq.npy", bottleneck_features_train)
    print('done training')   

    
    #VALIDATION##################################################################################
    
    generator_val = datagen.flow_from_dataframe(test_df, directory=None, x_col='filename', y_col='class', target_size=(img_width, img_height),batch_size=batch_size, 
                    class_mode='categorical', shuffle=False) 
    print('generator_val is',generator_val)
    nb_validation_samples = len(test_df) 
    predict_size_validation = int(math.ceil(nb_validation_samples / batch_size)) 
    bottleneck_features_validation = inception_resnet_v2.predict_generator(generator_val, predict_size_validation) 
    np.save("features_validation_inception_fusion_seq.npy", bottleneck_features_validation)
    print('validation done')
    
    
    # Training data load ##############################################################################
    load_train = datagen.flow_from_dataframe(train_df, directory=None, x_col='filename', y_col='class', target_size =(img_width, img_height), batch_size=batch_size, 
                      class_mode = 'categorical', shuffle=False) 
    nb_train_samples = len(train_df)
    train_data = np.load("features_training_inception_fusion_seq.npy") 
    print ('train_data.shape',train_data.shape)
    train_labels = tr_img_Y  
    train_labels = to_categorical(train_labels, num_classes=n_classes)
    #print('train_labels',train_labels)
    
    # validation data load ##########################################################################################
    load_val = datagen.flow_from_dataframe(test_df, directory=None, x_col='filename', y_col='class', target_size=(img_width, img_height),batch_size=batch_size, 
                    class_mode='categorical', shuffle=False) 
    nb_validation_samples = len(test_df)
    validation_data = np.load("features_validation_inception_fusion_seq.npy") 
    validation_labels = ts_img_Y  
    validation_labels = to_categorical(validation_labels, num_classes=n_classes)
    z=[train_data, validation_data, validation_labels, train_labels]
    return z
im=images()  ## stores train_data, validation_data, validation_labels, train_labels

def create_dense(feed):
# define our MLP network
    #global trainRX
    model = Sequential()
    model.add(Dense(64, input_shape=feed, activation='relu', name='fc1')) ### input shape to dense is 1D
    model.add(BatchNormalization())## add after Activation function ####
    model.add(Dense(32, activation='relu', name='fc2'))
    model.add(BatchNormalization())
    return model

def create_vgg(input_shape, cls, optimizer='adam', fine_tune=0):
    ######### Adding own model on top of vgg ##################
    model = Sequential() 
    model.add(Flatten(input_shape=input_shape)) 
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(layers.LeakyReLU(alpha=0.3))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(50))
    model.add(layers.LeakyReLU(alpha=0.3))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))##### try 0.7 dropout
    #model.add(Dense(n_classes, activation='softmax'))
    #model.add(Dense(32))
    #model.summary()
    # Group the convolutional base and new fully-connected layers into a Model object.
    return model


input_dense=[]
train_data=im[0]
val=im[1]
val_label=im[2]
tr_label=im[3]
input_dense=tr_rough_X[0].shape



Found 220 validated image filenames belonging to 11 classes.
nb_train_samples 220
done training
Found 55 validated image filenames belonging to 11 classes.
generator_val is <tensorflow.python.keras.preprocessing.image.DataFrameIterator object at 0x00000213975BCFA0>
validation done
Found 220 validated image filenames belonging to 11 classes.
train_data.shape (220, 2, 14, 1536)
Found 55 validated image filenames belonging to 11 classes.


In [7]:

tr_rough_X_tensor=tf.convert_to_tensor(tr_rough_X, np.float32)
tr_rough_Y_tensor=tf.convert_to_tensor(tr_rough_Y, np.float32)
ts_rough_X_tensor=tf.convert_to_tensor(ts_rough_X, np.float32)
ts_rough_Y_tensor=tf.convert_to_tensor(ts_rough_Y, np.float32)

class_names=['ABS','PA','PC','PP','WOOD1','WOOD2','WOOD3','WOOD4','WOOD5','WOOD6','WOOD7']
mlp = create_dense(input_dense)
cnn = create_vgg(train_data.shape[1:],n_classes)
combinedInput = concatenate([mlp.output, cnn.output])
top= Dense(64, activation="relu")(combinedInput)## keep neurons in multiples of 4
top_model= Dense(11, activation="softmax")(top)
model = Model(inputs=[mlp.input, cnn.input], outputs=top_model)
## training ##################################################################################
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-4),metrics=['acc'])

history= model.fit(x=[tr_rough_X_tensor, train_data], y=tr_rough_Y_tensor, validation_data=([ts_rough_X_tensor, val], 
                   ts_rough_Y_tensor), epochs=100, batch_size=32)
(eval_loss, eval_accuracy) = model.evaluate([ts_rough_X_tensor, val], ts_rough_Y_tensor, batch_size=batch_size, verbose=1)

print("test accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("test Loss: {}".format(eval_loss)) 
model.save('fusion_model_inception_seq.h5')


Epoch 1/100
7/7 [==============================] - 0s 56ms/step - loss: 2.8333 - acc: 0.1091 - val_loss: 4.5567 - val_acc: 0.0909
Epoch 2/100
7/7 [==============================] - 0s 29ms/step - loss: 2.4636 - acc: 0.1818 - val_loss: 3.5501 - val_acc: 0.0545
Epoch 3/100
7/7 [==============================] - 0s 30ms/step - loss: 2.2808 - acc: 0.2091 - val_loss: 2.9460 - val_acc: 0.0727
Epoch 4/100
7/7 [==============================] - 0s 31ms/step - loss: 2.2437 - acc: 0.2136 - val_loss: 2.6003 - val_acc: 0.1273
Epoch 5/100
7/7 [==============================] - 0s 29ms/step - loss: 2.0962 - acc: 0.2545 - val_loss: 2.4202 - val_acc: 0.1273
Epoch 6/100
7/7 [==============================] - 0s 29ms/step - loss: 1.9479 - acc: 0.2909 - val_loss: 2.2781 - val_acc: 0.1091
Epoch 7/100
7/7 [==============================] - 0s 28ms/step - loss: 1.7392 - acc: 0.3727 - val_loss: 2.1651 - val_acc: 0.1636
Epoch 8/100
7/7 [==============================] - 0s 30ms/step - loss: 1.7428 - acc: 0.37

7/7 [==============================] - 0s 30ms/step - loss: 0.3099 - acc: 0.9682 - val_loss: 0.7216 - val_acc: 0.8000
Epoch 64/100
7/7 [==============================] - 0s 28ms/step - loss: 0.3147 - acc: 0.9773 - val_loss: 0.7154 - val_acc: 0.8000
Epoch 65/100
7/7 [==============================] - 0s 28ms/step - loss: 0.2930 - acc: 0.9909 - val_loss: 0.7128 - val_acc: 0.7818
Epoch 66/100
7/7 [==============================] - 0s 29ms/step - loss: 0.2805 - acc: 0.9909 - val_loss: 0.7106 - val_acc: 0.7818
Epoch 67/100
7/7 [==============================] - 0s 32ms/step - loss: 0.3343 - acc: 0.9636 - val_loss: 0.7060 - val_acc: 0.7818
Epoch 68/100
7/7 [==============================] - 0s 28ms/step - loss: 0.2919 - acc: 0.9864 - val_loss: 0.6988 - val_acc: 0.7818
Epoch 69/100
7/7 [==============================] - 0s 31ms/step - loss: 0.2820 - acc: 0.9682 - val_loss: 0.6835 - val_acc: 0.8000
Epoch 70/100
7/7 [==============================] - 0s 34ms/step - loss: 0.2672 - acc: 0.9818 - 

In [ ]:
##### ACCURACY ######################
ax = plt.axes()
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Test acc')
plt.title('Training and Test Accuracy',fontsize=20)
plt.ylabel('Accuracy',fontsize=20) 
plt.xlabel('Epoch',fontsize=20)
plt.legend()
plt.tick_params(axis='both', labelsize=12)
plt.savefig("fusion_inception_ACCgraph.pdf",bbox_inches='tight')
plt.savefig("fusion_inception_ACCgraph.eps",bbox_inches='tight')

###### LOSS ################################
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Test loss')
plt.title('Training and Test Loss',fontsize=20)
plt.ylabel('Loss',fontsize=20) 
plt.xlabel('Epoch',fontsize=20)
plt.legend()
plt.tick_params(axis='both', labelsize=12)
plt.savefig("fusion_inception_loss.pdf",bbox_inches='tight')
plt.savefig("fusion_inception_loss.eps",bbox_inches='tight')

In [ ]:
y_pred=model.predict([ts_rough_X_tensor, val]) 
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(ts_rough_Y_tensor, axis=1)
cm = confusion_matrix(y_test, y_pred)
#print(cm)
import seaborn as sns; sns.set_theme()
sn.set(font_scale=2)
graph=plt.figure(figsize = (10,10))
classes=['ABS','PA','PC','PP','WOOD1','WOOD2','WOOD3','WOOD4','WOOD5','WOOD6','WOOD7']
ax = sn.heatmap(cm, cmap="Blues", annot=True, xticklabels=classes, yticklabels=classes, cbar=True)
ax.set(title='Confusion Matrix', xlabel="Predicted labels", ylabel="True labels")
plt.xticks(rotation=45)
plt.tick_params(axis='both', labelsize=18)
plt.show()
graph.savefig("fusion_inception_confusion_matrix.pdf",bbox_inches='tight')###################
graph.savefig("fusion_inception_confusion_matrix.eps",bbox_inches='tight')
####### generate classification report ###############################################################
report=classification_report(y_test, y_pred)
print(report)
report_df = pd.DataFrame([report])
print(report_df.to_csv('fusion_inception_report.csv')) 